Optimizing shipment to different regions. Function takes in any input file and ouputs objective values

In [50]:
from gurobipy import Model, GRB
import pandas as pd
def optimizeShipment(inputFile,outputFile):
    # Reading in data
    fcs=pd.read_excel(inputFile,sheet_name='Fulfilment Centers',index_col=0)
    regions=pd.read_excel(inputFile,sheet_name='Regions',index_col=0)
    distances=pd.read_excel(inputFile,sheet_name='Distances',index_col=0)
    items=pd.read_excel(inputFile,sheet_name='Items',index_col=0)
    demand=pd.read_excel(inputFile,sheet_name='Demand',index_col=0)
    I=fcs.index
    J=regions.index
    K=items.index
    q=fcs['capacity']
    delta=distances.T
    w=items['shipping_weight']
    s=items['storage_size']
    d=demand.T
    
    # Constructing the formulation
    mod=Model()
    x=mod.addVars(I,J,K,name='x')
    mod.setObjective(1.38*sum(w.loc[k]*delta.loc[i,j]*x[i,j,k] for i in I for j in J for k in K))
    for i in I:
        mod.addConstr(sum(s.loc[k]*x[i,j,k] for j in J for k in K)<=q.loc[i],name=f'Capacity_{i}')
    for j in J:
        for k in K:
            mod.addConstr(sum(x[i,j,k] for i in I)>=d.loc[j,k],name=f'Demand_{j}_{k}')
    mod.setParam('OutputFlag',False)
    mod.optimize()
    print('Finished optimizing! Objective value:',mod.objval)
    
    # Write your code to write to outputFile below
    writer=pd.ExcelWriter(outputFile)
    pd.DataFrame([mod.objval],columns=['Minimum Total Cost'])\
        .to_excel(writer,sheet_name='Summary',index=False)
    l = []
    for i in I:
        for j in J:
            for k in K:
                if x[i,j,k].x>0:
                    l.append([i, j, k, x[i,j,k].x])
    df = pd.DataFrame(l, columns = ['FC_name', 'region_ID', 'item_ID', 'Shipment'])
    df.to_excel(writer, sheet_name='Solution', index = False)
    writer.save()
    
    
    
    
    

In [51]:
# Test code
optimizeShipment('12-retail-toy-input.xlsx','12-retail-test-output.xlsx')

Finished optimizing! Objective value: 3400.769189999999


In [49]:
# Test code (might take a minute to run)
optimizeShipment('12-retail-real-input.xlsx','12-retail-real-output.xlsx')

Finished optimizing! Objective value: 9841229.288170151
